# VannaAI

Es una librería de python que permite crear código SQL a partir de texto usando algún modelo de AI.

Esto se hace mediante un arquitectura llamada RAG, la cual requiere mínimo:

- Base de datos vectorizada: donde se almacena la documentación relevante para el contexto.
- LLM: lo que en este caso crea el código SQL usando la documentación obtenida del elemento anterior.



GitHub: https://github.com/vanna-ai/vanna

Docs: https://vanna.ai/docs/index.html


In [ ]:
install vanna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 35.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.2/213.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 3.9 MB/s eta 0:00:00
  Created wheel for flasgger: filename=flasgger-0.9.7.1-py2.py3-none-any.whl size=3865185 sha256=d397da86c16f59cd95b20616fe5aea7de47914cf00fbe825fa717910747c9fd9
  Stored in directory: /root/.cache/pip/wheels/da/2d/f6/c125d4b8701ac49fdb2abbfb978036c46d5ad64d2ffaa2ac34
Successfully built flasgger


In [ ]:
import sqlite3
import pandas as pd
from vanna.remote import VannaDefault

from dotenv import load_dotenv
import os


In [ ]:

load_dotenv(dotenv_path='.env')

# https://vanna.ai/account/profile

api_key = os.getenv('RAPIDAPI_KEY')
vanna_model_name = os.getenv('RAPIDAPI_HOST')
vn = VannaDefault(model=vanna_model_name, api_key=api_key)

Para "entrenar" podemos utilizar 3 cosas:

- DDL: definición de las tablas.
- Question/SQL: Pregunta con su respectiva respuesta en SQL.
- Documentación: contexto adicional a modo de texto.

In [ ]:
vn.train(
    ddl="""
CREATE TABLE clientes (
    cliente_id    INTEGER PRIMARY KEY,
    nombre        TEXT    NOT NULL,
    ciudad        TEXT,
    edad          INTEGER,
    fecha_registro DATE
)
"""
)

Adding ddl: 
CREATE TABLE clientes (
    cliente_id    INTEGER PRIMARY KEY,
    nombre        TEXT    NOT NULL,
    ciudad        TEXT,
    edad          INTEGER,
    fecha_registro DATE
)



'14743677-ddl'

In [ ]:
vn.train(
    ddl="""
CREATE TABLE pedidos (
    pedido_id      INTEGER PRIMARY KEY,
    cliente_id     INTEGER,
    producto       TEXT    NOT NULL,
    cantidad       INTEGER NOT NULL,
    precio_unit    REAL    NOT NULL,
    fecha_pedido   DATE    NOT NULL,
    estado         TEXT    NOT NULL,
    FOREIGN KEY(cliente_id) REFERENCES clientes(cliente_id)
)
"""
)

Adding ddl: 
CREATE TABLE pedidos (
    pedido_id      INTEGER PRIMARY KEY,
    cliente_id     INTEGER,
    producto       TEXT    NOT NULL,
    cantidad       INTEGER NOT NULL,
    precio_unit    REAL    NOT NULL,
    fecha_pedido   DATE    NOT NULL,
    estado         TEXT    NOT NULL,
    FOREIGN KEY(cliente_id) REFERENCES clientes(cliente_id)
)



'14743678-ddl'

In [ ]:
vn.train(
    documentation="El usuario puede usar el nombre de orden en lugar de pedido."
)

Adding documentation....


'3431227-doc'

In [ ]:
vn.train(
    documentation="Los posibles estados de un pedido son: ENVIADO, ENTREGADO, PENDIENTE, CANCELADO."
)

Adding documentation....


'3431228-doc'

In [ ]:
vn.train(
    question="Cómo se clasifica el nivel de gasto de cada cliente en “ALTO”, “MEDIO” o “BAJO”, según el total que ha gastado?",
    sql="""
WITH gastos AS (
  SELECT cliente_id,
         SUM(cantidad * precio_unit) AS monto_total
    FROM pedidos
   GROUP BY cliente_id
)
SELECT c.nombre,
       g.monto_total,
       CASE
         WHEN g.monto_total > 1000 THEN 'ALTO'
         WHEN g.monto_total BETWEEN 300 AND 1000 THEN 'MEDIO'
         ELSE 'BAJO'
       END AS categoria
  FROM gastos AS g
  JOIN clientes AS c ON c.cliente_id = g.cliente_id;
"""
)

'254ebadffd85e71abcb97cae617348bb-sql'

Para visualizar lo almacenado hasta el momento:

In [ ]:
vn.get_training_data()

,id,training_data_type,question,content
0,1218648-sql,sql,Cómo se clasifica el nivel de gasto de cada cl...,"\nWITH gastos AS (\n SELECT cliente_id,\n ..."
1,14743678-ddl,ddl,None,\nCREATE TABLE pedidos (\n pedido_id I...
2,3431228-doc,documentation,None,Los posibles estados de un pedido son: ENVIADO...
3,14743677-ddl,ddl,None,\nCREATE TABLE clientes (\n cliente_id I...
4,3431227-doc,documentation,None,El usuario puede usar el nombre de orden en lu...


Para eliminar algo de la base de datos:

In [ ]:
vn.remove_training_data('') # se ingresa el id que se visualiza en la tabla anterior

True

Para generar SQL:

In [ ]:
query = vn.generate_sql(
    "cual fue la ultima orden pedida?"
)

SQL Prompt: [{'role': 'system', 'content': "You are a SQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \n\nCREATE TABLE pedidos (\n    pedido_id      INTEGER PRIMARY KEY,\n    cliente_id     INTEGER,\n    producto       TEXT    NOT NULL,\n    cantidad       INTEGER NOT NULL,\n    precio_unit    REAL    NOT NULL,\n    fecha_pedido   DATE    NOT NULL,\n    estado         TEXT    NOT NULL,\n    FOREIGN KEY(cliente_id) REFERENCES clientes(cliente_id)\n)\n\n\n\nCREATE TABLE clientes (\n    cliente_id    INTEGER PRIMARY KEY,\n    nombre        TEXT    NOT NULL,\n    ciudad        TEXT,\n    edad          INTEGER,\n    fecha_registro DATE\n)\n\n\n\n===Additional Context \n\nEl usuario puede usar el nombre de orden en lugar de pedido.\n\nLos posibles estados de un pedido son: ENVIADO, ENTREGADO, PENDIENTE, CANCELADO.\n\n===Response Guidelines \n

In [ ]:
print(query)

SELECT *
FROM pedidos
ORDER BY fecha_pedido DESC
LIMIT 1;


In [ ]:
conn = sqlite3.connect('db_basic_queries.sqlite3')

In [ ]:
pd.read_sql_query(query, conn)

,pedido_id,cliente_id,producto,cantidad,precio_unit,fecha_pedido,estado
0,105,4,Smartphone,1,1200.0,2025-07-02,ENVIADO
